# Setup

In [1]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import pandas as pd
import os
from datetime import datetime

# Import specific packages
import re
from collections import Counter
from scipy.sparse import csr_matrix
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils import shuffle

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Load data

In [2]:
item_cat = pd.read_csv('./data/item_categories.csv')
items = pd.read_csv('./data/items.csv')
sales_train = pd.read_csv('./data/sales_train.csv')
shops = pd.read_csv('./data/shops.csv')
test = pd.read_csv('./data/test.csv')
sample_submission = pd.read_csv('./data/sample_submission.csv')

In [3]:
item_cat.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [4]:
items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [5]:
sales_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [6]:
shops.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [7]:
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [8]:
sample_submission.head()

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5


# Data wrangling

**Merge and flatten dataframe**

First of all, let's join the `items` and `item_cat` together.

In [9]:
items = pd.merge(items, item_cat, how='left', on=['item_category_id'])

**Apply tfidf transform on the `item_name`, `item_category_name` and `shop_name`**

Let convert a collection of raw documents to a matrix of TF-IDF features using the class `sklearn.feature_extraction.text.TfidfVectorizer`, which is equivalent to `CountVectorizer` followed by `TfidfTransformer`.

In [10]:
feature_cnt = 25
tfidf = TfidfVectorizer(max_features=feature_cnt)
item_name = pd.DataFrame(tfidf.fit_transform(items['item_name']).toarray())

In [11]:
item_name.head()

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.637898,0.0,0.0,...,0.0,0.403761,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.483839
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


Let's create a function to merge the items and item_name together.

In [12]:
def merge_dataframe(df_left, df_right, column_name_prefix):
    for column in df_right.columns.values:
        col = column_name_prefix + str(column)
        df_left[col] = df_right[column]

In [13]:
merge_dataframe(items, item_name, 'item_name')

In [14]:
items.head()

,item_name,item_id,item_category_id,item_category_name,item_name0,item_name1,item_name2,item_name3,item_name4,item_name5,...,item_name15,item_name16,item_name17,item_name18,item_name19,item_name20,item_name21,item_name22,item_name23,item_name24
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40,Кино - DVD,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,!ABBYY FineReader 12 Professional Edition Full...,1,76,Программы - Для дома и офиса (Цифра),0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.403761,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.483839
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40,Кино - DVD,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40,Кино - DVD,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,***КОРОБКА (СТЕКЛО) D,4,40,Кино - DVD,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


Let's do the same transformation to the item_category_name and shop_name

In [15]:
feature_cnt = 25
tfidf = TfidfVectorizer(max_features=feature_cnt)
item_cat_name = pd.DataFrame(tfidf.fit_transform(items['item_category_name']).toarray())

In [16]:
merge_dataframe(items, item_cat_name, 'item_cat_name')

In [17]:
feature_cnt = 25
tfidf = TfidfVectorizer(max_features=feature_cnt)
shop_name = pd.DataFrame(tfidf.fit_transform(shops['shop_name']).toarray())

In [18]:
merge_dataframe(shops, shop_name, 'shop_name')

As we notice that there are some negative item_price and negative item_cnt_day. We will filter out those records.

In [19]:
sales_train = sales_train[(sales_train['item_price']>0) & (sales_train['item_cnt_day']>0)]

In [20]:
print('num_records in sales_train:', len(sales_train))

num_records in sales_train: 2928492


Let's check if the (date, shop_id, item_id) are unique.

In [21]:
a = sales_train.groupby(['date', 'shop_id', 'item_id'], as_index=False)['item_cnt_day'].count().rename(columns={'item_cnt_day': 'count'}).sort_values(by=['count'], ascending=False)
a.head()

,date,shop_id,item_id,count
2674113,29.01.2013,25,12133,2
669764,07.10.2015,12,21619,2
2778335,30.03.2014,31,16875,2
1073110,12.02.2015,42,21619,2
1976537,21.11.2014,31,16587,2


In [22]:
sales_train[(sales_train['date']=='29.01.2013') & (sales_train['shop_id']==25) & (sales_train['item_id']==12133)]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
6959,29.01.2013,0,25,12133,889.0,1.0
6960,29.01.2013,0,25,12133,1389.0,1.0


Now that we know the same item could be sold at different price at the same day. Therefore there are two records. As in the test dataset, there is no item_price. Hence let's assume that the item_price of the month to be predicted to be the latest item_price.

**Generate table item_price_latest with the last item_price**

In [23]:
item_price_latest = sales_train.sort_values(by=['date'], ascending=False).groupby(['item_id', 'shop_id'], as_index=False)['item_price'].first()#.rename(columns={'item_price': 'item_price'})
item_price_latest.head()

,item_id,shop_id,item_price
0,0,54,58.0
1,1,55,4490.0
2,2,54,58.0
3,3,54,58.0
4,4,54,58.0


**Extract date columns to year, month, day ...**

Now let's extract the `date` column to: `year`, `month`, `day`, `day_of_year`, `weekday`.

In [24]:
sales_train['date'] = sales_train['date'].apply(lambda x: datetime.strptime(x, '%d.%m.%Y'))
sales_train['year'] = sales_train['date'].apply(lambda x: x.year)
sales_train['month'] = sales_train['date'].apply(lambda x: x.month)

**Aggregate on month level**

In [25]:
sales_train1 = sales_train.groupby(['shop_id', 'item_id', 'year', 'month'], as_index=False)['item_cnt_day'].sum().rename(columns={'item_cnt_day':'item_cnt_month'})

**Generate table item_price_avg for each shop and month**

As we need to predict the sales on month level, let's also aggregate the item_price on month level.

In [26]:
item_price_avg = sales_train.groupby(['item_id', 'shop_id', 'year', 'month'], as_index=False)['item_price'].mean()
item_price_avg.head()

,item_id,shop_id,year,month,item_price
0,0,54,2014,9,58.0
1,1,55,2014,4,4490.0
2,1,55,2014,7,4490.0
3,1,55,2014,8,4490.0
4,1,55,2014,9,4490.0


**Create training data**

In [27]:
sales_train2 = pd.merge(sales_train1, item_price_avg, how='left', on=['shop_id','item_id','year','month'])
sales_train3 = pd.merge(sales_train2, items, how='left', on=['item_id'])
sales_train4 = pd.merge(sales_train3, shops, how='left', on=['shop_id'])
train = sales_train4

In [28]:
train.head()

,shop_id,item_id,year,month,item_cnt_month,item_price,item_name,item_category_id,item_category_name,item_name0,...,shop_name15,shop_name16,shop_name17,shop_name18,shop_name19,shop_name20,shop_name21,shop_name22,shop_name23,shop_name24
0,0,30,2013,2,31.0,265.0,007: КООРДИНАТЫ «СКАЙФОЛЛ»,40,Кино - DVD,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0,31,2013,2,11.0,434.0,007: КООРДИНАТЫ «СКАЙФОЛЛ» (BD),37,Кино - Blu-Ray,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0,32,2013,1,6.0,221.0,1+1,40,Кино - DVD,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0,32,2013,2,10.0,221.0,1+1,40,Кино - DVD,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0,33,2013,1,3.0,347.0,1+1 (BD),37,Кино - Blu-Ray,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


**Create test data**

In [29]:
test2 = pd.merge(test, item_price_latest, how='left', on=['shop_id','item_id'])
test3 = pd.merge(test2, items, how='left', on=['item_id'])
test4 = pd.merge(test3, shops, how='left', on=['shop_id'])
df_test = test4

In [30]:
df_test.head()

,ID,shop_id,item_id,item_price,item_name,item_category_id,item_category_name,item_name0,item_name1,item_name2,...,shop_name15,shop_name16,shop_name17,shop_name18,shop_name19,shop_name20,shop_name21,shop_name22,shop_name23,shop_name24
0,0,5,5037,749.0,"NHL 15 [PS3, русские субтитры]",19,Игры - PS3,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1,5,5320,NaN,ONE DIRECTION Made In The A.M.,55,Музыка - CD локального производства,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,2,5,5233,599.0,"Need for Speed Rivals (Essentials) [PS3, русск...",19,Игры - PS3,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,3,5,5232,599.0,"Need for Speed Rivals (Classics) [Xbox 360, ру...",23,Игры - XBOX 360,0.0,0.588259,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,4,5,5268,NaN,"Need for Speed [PS4, русская версия]",20,Игры - PS4,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


let's add the month and year to the df_test as well.

In [31]:
train.sort_values(by=['year','month'], ascending=[False, False]).head(1)

,shop_id,item_id,year,month,item_cnt_month,item_price,item_name,item_category_id,item_category_name,item_name0,...,shop_name15,shop_name16,shop_name17,shop_name18,shop_name19,shop_name20,shop_name21,shop_name22,shop_name23,shop_name24
8113,2,31,2015,10,1.0,399.0,007: КООРДИНАТЫ «СКАЙФОЛЛ» (BD),37,Кино - Blu-Ray,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.49858,0.0,0.0,0.0


In [32]:
df_test['year'] = 2015
df_test['month'] = 11

Now we have two dataframe:

- **train**: this would be used for training the model. In the next section, we will spilt this to a training and testing set.
- **df_test**: this would be the month to be predicted.

## Train test split

In [33]:
train = shuffle(train, random_state=42)

X = train[[col for col in train.columns.values if col not in ['item_name', 'item_category_name', 'shop_name']]]

y = train['item_cnt_month']

len_X = len(X)
test_size = 0.25

X_train = X[:-int(len_X * test_size)]
y_train = y[:-int(len_X * test_size)]
X_test = X[-int(len_X * test_size):-1]
y_test = y[-int(len_X * test_size):-1]

# Train models

In [34]:
rf_reg = RandomForestRegressor(n_estimators=25, n_jobs=-1, max_depth=15, random_state=42)
rf_reg.fit(X_train, y_train)

y_train_pred = rf_reg.predict(X_train)
y_test_pred = rf_reg.predict(X_test)

mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)

print('mse_train:', mse_train)
print('mse_test:', mse_test)

mse_train: 0.2097904898977756
mse_test: 0.04698856823071475


## Optional: other methods of handling item_name

**Stemming**

Let's try some stemming! For this to work, we need the Natural Language Toolkit ([NLTK](http://www.nltk.org/)). 

In [35]:
try:
    import nltk
    
    stemmer = nltk.PorterStemmer()
    for word in ("Computations", "Computation", "Computing", "Computed", "Compute", "Compulsive"):
        print(word, '=>', stemmer.stem(word))
except ImportError:
    print("Error: stemming requires the NLTK module.")
    stemmer = None

Computations => comput
Computation => comput
Computing => comput
Computed => comput
Compute => comput
Compulsive => compuls


**Create custom transformer**

- TextToWordCounterTransformer
- WordCounterToVector

Now let's create a `custom transformer` that we will use to convert sentense to word counters.

In [36]:
from sklearn.base import BaseEstimator, TransformerMixin


class TextToWordCounterTransformer(BaseEstimator, TransformerMixin):
    
    def __init__(self, 
                 lower_case=True, 
                 remove_punctuation=True, 
                 replace_numbers=True, 
                 stemming=True):
        self.lower_case = lower_case
        self.remove_punctuation = remove_punctuation
        self.replace_numbers = replace_numbers
        self.stemming = stemming
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X_transformed = []
        for text in X:
            if self.lower_case:
                text = text.lower()
            if self.remove_punctuation:
                text = re.sub(r'\W+', ' ', text)
            if self.replace_numbers:
                text = re.sub(r'\d+(?:\.\d*(?:[eE]\d+))?', 'NUMBER', text)
            if self.stemming and stemmer is not None:
                text = stemmer.stem(text)
            word_counts = Counter(text.split())
            X_transformed.append(word_counts)
        return np.array(X_transformed)

In [40]:
class WordCounterToVector(BaseEstimator, TransformerMixin):
    
    def __init__(self, len_vocabulary=25):
        self.len_vocabulary = len_vocabulary
    def fit(self, X, y=None):
        vocabulary_counter = Counter({})
        for counter in X:
            vocabulary_counter = sum((vocabulary_counter, counter), Counter())
        vocabulary_counter = vocabulary_counter.most_common(self.len_vocabulary)
        self.vocabulary = {word: index for index, (word, count) in enumerate(vocabulary_counter, start=1)}
        return self
    def transform(self, X, y=None):
        rows = []
        cols = []
        data = []
        for row, word_count in enumerate(X):
            for key, val in word_count.items():
                rows.append(row)
                cols.append(self.vocabulary.get(key,0))
                data.append(val)
        a = csr_matrix((data, (rows, cols)), shape=(len(X),self.len_vocabulary+1))
        return a

Now let's create a proprocess pipeline to convert the `item_name` and `item_category_name` into word counter vector.

In [41]:
preprocess_pipeline_item_name = Pipeline((
    ('text_to_word_counter', TextToWordCounterTransformer()),
    ('word_counter_to_vector', WordCounterToVector())
))

In [42]:
preprocess_pipeline_category_name = Pipeline((
    ('text_to_word_counter', TextToWordCounterTransformer()),
    ('word_counter_to_vector', WordCounterToVector())
))